In [1]:
pip install tweepy praw python-dotenv vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14/14 [praw]2m13/14 [praw]sts-oauthlib]
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import tweepy
import praw
from dotenv import load_dotenv
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer        

In [4]:
os.environ["REDDIT_CLIENT_ID"] = "w8vAFuTjHB6TWiaB-5muLw"
os.environ["REDDIT_CLIENT_SECRET"] = "Wlalndnv58IvLGef453qYI8JUVT0iw"
os.environ["REDDIT_USER_AGENT"]='sentiment-demo/0.1 by ivan'

In [5]:
reddit = praw.Reddit(
    client_id=os.environ["REDDIT_CLIENT_ID"],
    client_secret=os.environ["REDDIT_CLIENT_SECRET"],
    user_agent=os.environ["REDDIT_USER_AGENT"],
)
reddit.read_only = True

# 3) Sanity check: a very simple call
try:
    print("Read-only:", reddit.read_only)
    it = reddit.subreddit("python").hot(limit=1)
    print(next(it).title)
    print("OK ✅")
except Exception as e:
    print("Failed ❌ ->", type(e).__name__, e)

Read-only: True
Sunday Daily Thread: What's everyone working on this week?
OK ✅


In [6]:
sub = reddit.subreddit("stocks")   # choose any subreddit

# Top N new posts matching a keyword
results = []
for post in sub.search(query="nvidia", sort="new", limit=25):
    results.append({
        "id": post.id,
        "title": post.title,
        "score": post.score,
        "created_utc": post.created_utc,
        "url": post.url,
        "num_comments": post.num_comments,
        "selftext": post.selftext[:500],  # preview
    })

print(f"Fetched {len(results)} posts")
print(results[:2])

Fetched 25 posts
[{'id': '1n21zmh', 'title': 'after Nvidia’s Q2 earnings, is growth slowing or just market overreaction?', 'score': 1, 'created_utc': 1756352588.0, 'url': 'https://www.reddit.com/r/stocks/comments/1n21zmh/after_nvidias_q2_earnings_is_growth_slowing_or/', 'num_comments': 21, 'selftext': 'After Nvidia released its Q2 earnings, the stock fell by up to 5%. While the earnings themselves were still strong and even exceeded some expectations, the market seems to be more cautious about the future, raising the bar for performance. Does this reaction indicate that Nvidia’s growth is slowing down, or is it just a short-term market sentiment? In this situation, would you still choose to hold a large position in Nvidia, or consider adjusting your holdings to manage risk?'}, {'id': '1n1y5cm', 'title': 'Nvidia CEO Huang says bringing Blackwell AI chip to China ‘is a real possibility’', 'score': 127, 'created_utc': 1756341447.0, 'url': 'https://www.reddit.com/r/stocks/comments/1n1y5cm/

In [7]:
if results:
    post = reddit.submission(id=results[0]["id"])
    post.comments.replace_more(limit=0)  # flatten "MoreComments"
    comments = [c.body for c in post.comments.list()[:50]]  # first 50
    print(f"Collected {len(comments)} comments")
    print(comments[:3])

Collected 21 comments
['56% up year over year. Definitely not slowing down.', 'This is the perfect time to buy an actual dip\n\nIf they’re start selling in China officially - that’s another $50B in revenue a year', "Doesn't sound you like you paid attention to the guidance or earnings call. \n\nQ2 & Q3 Guidance includes $0 for H20\n\nWhen China opens back up, Nvidia will go up"]


In [8]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def score(text):
    s = analyzer.polarity_scores(text)
    return s["compound"]  # -1 (neg) .. +1 (pos)

# Example: score tweets
if 'all_rows' in globals() and all_rows:
    for r in all_rows[:10]:
        r["sentiment"] = score(r["text"])
    print(all_rows[:3])

# Example: score reddit comments
if 'comments' in globals() and comments:
    scored = [{"text": c[:120]+"...", "sentiment": score(c)} for c in comments[:10]]
    print(scored[:3])

[{'text': '56% up year over year. Definitely not slowing down....', 'sentiment': 0.4019}, {'text': 'This is the perfect time to buy an actual dip\n\nIf they’re start selling in China officially - that’s another $50B in rev...', 'sentiment': 0.5719}, {'text': "Doesn't sound you like you paid attention to the guidance or earnings call. \n\nQ2 & Q3 Guidance includes $0 for H20\n\nWhen...", 'sentiment': -0.2755}]
